In [5]:
import time
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
#import talib
import pandas_ta as ta
from datetime import datetime
from time import sleep
from numba import jit
import numpy as np
import heapq
import warnings
warnings.filterwarnings('ignore')
from lightweight_charts import JupyterChart
import webbrowser
import plotly.graph_objects as go
import plotly.io as pio
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from scipy.stats import linregress
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [6]:
def calculate_pivot_points(df):
    pivot_points = []
    bottom_pivot_line = []  # Initialize as a list
    for date, group in df.groupby(df['datetime'].dt.date):
        if len(group) >= 0:
            prev_day_data = df[df['datetime'].dt.date < date].tail(125)
            if not prev_day_data.empty:
                pivot_point = (prev_day_data['high'].max() + prev_day_data['low'].min() + prev_day_data['close'].iloc[-1]) / 3
                pivot_points.extend([pivot_point] * len(group))

                bottom_pivot_line_value = (prev_day_data["high"].max() + prev_day_data["low"].min()) / 2
                bottom_pivot_line.extend([bottom_pivot_line_value] * len(group))  # Extend the list with the value
            else:
                pivot_points.extend([None] * len(group))
                bottom_pivot_line.extend([None] * len(group))
        else:
            pivot_points.extend([None] * len(group))
            bottom_pivot_line.extend([None] * len(group))

    df['Pivot Point'] = pivot_points
    df['Bottom Pivot Line'] = bottom_pivot_line

    return df




In [7]:
def pivot_getting(data):
    res1 = []
    sup1 = []
    res2 = []
    sup2 = []
    top_pivot_line = []
  
    for date, group in data.groupby(data['datetime'].dt.date):
        if len(group) >= 0:
            prev_day_data = data[data['datetime'].dt.date < date].tail(125)
            present_day_data = data[data['datetime'].dt.date == date]
            if len(prev_day_data) == 125:
                res1_val = (2 * present_day_data["Pivot Point"].max()) - prev_day_data['low'].min()         
                sup1_val = (2 * present_day_data["Pivot Point"].max()) - prev_day_data['high'].max()
                res2_val = present_day_data["Pivot Point"].max() + (prev_day_data['high'].max() - prev_day_data['low'].min())
                sup2_val = present_day_data["Pivot Point"].max() - (prev_day_data['high'].max() - prev_day_data['low'].min())
                top_val = present_day_data["Pivot Point"].max() + (present_day_data["Pivot Point"].max() - present_day_data["Bottom Pivot Line"].max())             
                res1.extend([res1_val] * len(group))
                sup1.extend([sup1_val] * len(group))
                res2.extend([res2_val] * len(group))
                sup2.extend([sup2_val] * len(group))
                top_pivot_line.extend([top_val] * len(group))
            else:
                res1.extend([None] * len(group))
                sup1.extend([None] * len(group))
                res2.extend([None] * len(group))
                sup2.extend([None] * len(group))
                top_pivot_line.extend([None] * len(group))
        else:
            res1.extend([None] * len(group))
            sup1.extend([None] * len(group))
            res2.extend([None] * len(group))
            sup2.extend([None] * len(group))
            top_pivot_line.extend([None] * len(group))
            
    data['res1'] = res1
    data['sup1'] = sup1
    data['res2'] = res2
    data['sup2'] = sup2
    data["top_pivot_line"] = top_pivot_line
    return data

In [8]:
def res3sup3(data):
    res3 = []
    sup3 = []

    for date, group in data.groupby(data['datetime'].dt.date):
        if len(group) >= 0:
            prev_day_data = data[data['datetime'].dt.date < date].tail(125)
            present_day_data = data[data['datetime'].dt.date == date]
            if len(prev_day_data) == 125:
                res3_val = prev_day_data["res1"].max() + (prev_day_data["high"].max() - prev_day_data['low'].min())         
                sup3_val = prev_day_data["sup1"].max() - (prev_day_data["high"].max() - prev_day_data['low'].min())
                  
                res3.extend([res3_val] * len(group))
                sup3.extend([sup3_val] * len(group))
                
            else:
                res3.extend([None] * len(group))
                sup3.extend([None] * len(group))
                
        else:
            res3.extend([None] * len(group))
            sup3.extend([None] * len(group))
            
            
    data['res3'] = res3
    data['sup3'] = sup3
    
    return data

In [9]:
@jit
def identify_price_position_numpy(open_prices, res2, res1, top_pivot_line, pivot_point, bottom_pivot_line, sup1, sup2,res3,sup3):
    upper_bound = []
    lower_bound = []
    for i in range(len(open_prices)):
        if open_prices[i]> res3[i]:
            upper_bound.append(res3[i] + res3[i]*0.05)
            lower_bound.append(res3[i])
        elif open_prices[i] > res2[i] and open_prices[i] < res3[i]:
            upper_bound.append(res3[i])
            lower_bound.append(res2[i])
        elif open_prices[i] < res2[i] and open_prices[i] > res1[i]:
            upper_bound.append(res2[i])
            lower_bound.append(res1[i])
        elif open_prices[i] < res1[i] and open_prices[i] > top_pivot_line[i]:
            upper_bound.append(res1[i])
            lower_bound.append(top_pivot_line[i])
        elif open_prices[i] < top_pivot_line[i] and open_prices[i] > pivot_point[i]:
            upper_bound.append(top_pivot_line[i])
            lower_bound.append(pivot_point[i])
        elif open_prices[i] < pivot_point[i] and open_prices[i] > bottom_pivot_line[i]:
            upper_bound.append(pivot_point[i])
            lower_bound.append(bottom_pivot_line[i])
        elif open_prices[i] < bottom_pivot_line[i] and open_prices[i] > sup1[i]:
            upper_bound.append(bottom_pivot_line[i])
            lower_bound.append(sup1[i])
        elif open_prices[i] < sup1[i] and open_prices[i] > sup2[i]:
            upper_bound.append(sup1[i])
            lower_bound.append(sup2[i])
        elif open_prices[i] < sup3[i] and open_prices[i] > sup2[i]:
            upper_bound.append(sup2[i])
            lower_bound.append(sup3[i])
        else:
            upper_bound.append(sup3[i])
            lower_bound.append(sup3[i] - sup3[i]*0.05)
    return upper_bound, lower_bound


In [10]:
@jit
def macd_(macd, macd_signal, macd_hist, upper_bound, lower_bound, close):
    signal = np.zeros(len(macd))
    
    for i in range(len(macd)):
        if macd[i] > macd_signal[i]  and lower_bound[i] < close[i] < upper_bound[i]:
            signal[i] = 1
        elif macd[i] > macd_signal[i] and close[i] >= upper_bound[i]:
            signal[i] = 0
        elif macd[i] < macd_signal[i]  and lower_bound[i] < close[i] < upper_bound[i]:
            signal[i] = -1
        elif macd[i] < macd_signal[i] and close[i] <= lower_bound[i]:
            signal[i] = 0
    
    return signal




In [11]:
@jit
def supertrend_y(data,lower_bound,upper_bound,close):
    supertrend1_y = []
    for i in range(len(data)):
        if data["supertrend"][i] > data["close"][i]:
            supertrend1_y.append(-1)
        elif data["supertrend"][i] < data["close"][i]:
            supertrend1_y.append(1)
        else:
            supertrend1_y.append(0)
    return supertrend1_y

In [12]:
@jit
def bearish_divergence_rsi_macdxs(rsi_values, price_highs, lockback_period=25):
    signal1 = []
    bearish_divergence_observed = False  # Flag to track bearish divergence
    bearish_divergence_confirmed = False  # Flag to track confirmed bearish divergence
    
    for i in range(len(rsi_values)):
        if i >= lockback_period - 1:
            rsi_values_window = rsi_values[i-(lockback_period-1):]
            price_highs_window = price_highs[i-(lockback_period-1):]
            two_highest_highs = heapq.nlargest(2, zip(price_highs_window, range(len(price_highs_window))))
            highest_highs_indexes = [index for _, index in two_highest_highs]
            rsi_values_highest_highs = [rsi_values_window[index] for index in highest_highs_indexes]
            rsi_corresponding_To_max_price = rsi_values_highest_highs[0]
            rsi_corresponding_to_second_max = rsi_values_highest_highs[1]
            
            if rsi_corresponding_To_max_price < rsi_corresponding_to_second_max:
                if (65 <= min(rsi_values_highest_highs) <= 85 and 65 <= max(rsi_values_highest_highs) <= 85):
                    if abs(highest_highs_indexes[1] - highest_highs_indexes[0]) >= 4:
                        signal1.append(-1)
                        bearish_divergence_observed = True
                    else:
                        signal1.append(0)
                else:
                    signal1.append(0)
            else:
                signal1.append(0)
        else:
            signal1.append(0)
    
    return signal1

In [13]:
# Optimize parameters
ema_fast = 7  # Adjust this value
ema_slow = 21  # Adjust this value
rsi_overbought = 70  # Adjust this value
rsi_oversold = 30  # Adjust this value

@jit
def DEMA_Y(dema7, dema21, macd_hist, upper_bound, lower_bound, close, rsi_14, ema_fast, ema_slow, rsi_overbought, rsi_oversold):
    signal = []
    for i in range(len(dema21)):
        if (
            dema7[i] > dema21[i]
            and lower_bound[i] < close[i] < upper_bound[i]
            and rsi_14[i] < rsi_overbought
        ):
            signal.append(1)
        elif (
            dema7[i] > dema21[i]
            and close[i] >= upper_bound[i]
            and rsi_14[i] < rsi_overbought
        ):
            signal.append(0)
        elif (
            dema7[i] < dema21[i]
            and lower_bound[i] < close[i] < upper_bound[i]
            and rsi_14[i] > rsi_oversold
        ):
            signal.append(-1)
        elif (
            dema7[i] < dema21[i]
            and close[i] <= lower_bound[i]
            and rsi_14[i] > rsi_oversold
        ):
            signal.append(0)
        else:
            signal.append(0)
    return signal


In [14]:
@jit
def TEMA_Y(tema_7,tema_21,macd_hist,upper_bound,lower_bound,close):
    signal = []
    for i in range(len(tema_21)):
        if tema_7[i]>tema_21[i]  and lower_bound[i] < close[i] < upper_bound[i]:
            signal.append(1)
        elif tema_7[i] > tema_21[i] and close[i] >= upper_bound[i]:
            signal.append(0)
        elif tema_7[i]<tema_21[i]  and lower_bound[i] < close[i] < upper_bound[i]:
            signal.append(-1)
        elif tema_7[i] < tema_21[i] and close[i] <= lower_bound[i]:
            signal.append(0)
        else:
            signal.append(0)
    return signal



In [15]:
import pandas as pd
import ta  # Assuming you've imported the necessary libraries

def calculate_signals_adx1(df):

    bb_lower, bb_middle, bb_upper = ta.bbands(df["close"], length=20, std=2, mamode="sma")

    # Buy signals
    buy_condition = (df['ema9'] > df['ema26']) & (df['ema12'] > df['ema26']) & \
                    (df['adx1'] > 25) & (df['macd'] > df['macd_signal']) & \
                    (df['close'] > bb_middle)

    # Sell signals
    sell_condition = (df['ema9'] < df['ema26']) & (df['ema12'] < df['ema26']) & \
                     (df['adx1'] > 25) & (df['macd'] < df['macd_signal']) & \
                     (df['close'] < bb_middle)

    # Combine buy and sell signals
    signals = pd.DataFrame(index=df.index)
    signals['signals'] = 0
    signals.loc[buy_condition, 'signals'] = 1
    signals.loc[sell_condition, 'signals'] = -1

    return signals


In [16]:
def calculate_signals_adx2(close, ema9, ema26,rsi_values, macd, macd_signal,high,low,adx2):
   
    
    adx,a,b = ta.adx(df['high'], df['low'], df['close'], length=14)
    
    signals = np.zeros(len(close))
    
    for i in range(1, len(close)):
        if (
            (ema9[i] > ema26[i]) and
            (adx2[i] > 10) and  # Lowered ADX threshold
            (rsi_values[i] > 30) and  # Adjusted RSI threshold
            (macd[i] > macd_signal[i]) and
            (macd[i-1] <= macd_signal[i-1])
        ):
            signals[i] = 1  # Buy signal
        elif (
            (ema9[i] < ema26[i]) and
            (adx2[i] > 10) and  # Lowered ADX threshold
            (rsi_values[i] < 70) and  # Adjusted RSI threshold
            (macd[i] < macd_signal[i]) and
            (macd[i-1] >= macd_signal[i-1])
        ):
            signals[i] = -1  # Sell signal
            
    return signals

In [17]:
def generate_signals(data,upper_bound,lower_bound,close):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['close']
    
    # Initialize columns for DMplus, DMminus, TR, DIplus, DIminus, and ADX
    data['DMplus'] = 0 
    data['DMminus'] = 0
    data['TR'] = 0
    data['DIplus'] = 0
    data['DIminus'] = 0
    data['ADX'] = 0
    
    # Loop through each row of the data frame to calculate the values
    for i in range(1, len(data)):
        # Calculate DMplus
        if (data['high'].iloc[i] - data['high'].iloc[i - 1]) > (data['low'].iloc[i - 1] - data['low'].iloc[i]):
            data.at[data.index[i], 'DMplus'] = max(data['high'].iloc[i] - data['high'].iloc[i - 1], 0)
        
        # Calculate DMminus
        if (data['low'].iloc[i - 1] - data['low'].iloc[i]) > (data['high'].iloc[i] - data['high'].iloc[i - 1]):
            data.at[data.index[i], 'DMminus'] = max(data['low'].iloc[i - 1] - data['low'].iloc[i], 0)
        
        # Calculate TR
        data.at[data.index[i], 'TR'] = max(data['high'].iloc[i] - data['low'].iloc[i], 
                                            max(abs(data['high'].iloc[i] - data['close'].iloc[i - 1]), 
                                                abs(data['low'].iloc[i] - data['close'].iloc[i - 1])))
    
    # Calculate ADX
    for i in range(13, len(data)):
        data.at[data.index[i], 'ADX'] = (data['DMplus'].iloc[i - 13:i + 1].mean() / 
                                          data['TR'].iloc[i - 13:i + 1].mean()) * 100
    
    # Calculate DIplus and DIminus
    for i in range(13, len(data)):
        data.at[data.index[i], 'DIplus'] = (data['DMplus'].iloc[i - 13:i + 1].mean() / 
                                             data['TR'].iloc[i - 13:i + 1].mean()) * 100
        data.at[data.index[i], 'DIminus'] = (data['DMminus'].iloc[i - 13:i + 1].mean() / 
                                              data['TR'].iloc[i - 13:i + 1].mean()) * 100
    
    # Generate signals
    signals['Buy'] = 0
    signals['Sell'] = 0
    for i in range(14, len(data)):
        if (data['DIplus'].iloc[i] > data['DIminus'].iloc[i]) and (data['ADX'].iloc[i] > 20) and (lower_bound[i] < close[i] < upper_bound[i]):
            signals.at[data.index[i], 'Buy'] = 1
            
        elif (data['DIplus'].iloc[i] > data['DIminus'].iloc[i]) and (data['ADX'].iloc[i] > 20) and (close[i] >= upper_bound[i]):
            signals.at[data.index[i], 'Buy'] = 0
        
        elif (data['DIminus'].iloc[i] > data['DIplus'].iloc[i]) and (data['ADX'].iloc[i] > 20) and (lower_bound[i] < close[i] < upper_bound[i]):
            signals.at[data.index[i], 'Sell'] = 1

        elif (data['DIminus'].iloc[i] > data['DIplus'].iloc[i]) and (data['ADX'].iloc[i] > 20) and (close[i] <= lower_bound[i]):
            signals.at[data.index[i], 'Sell'] = 0
    return signals



In [18]:
def HA(df):
    df['HA_Close']=(df['open']+ df['high']+ df['low']+ df['close'])/4
    df['HA_Open']=(df['open']+df['close'])/2   
    #df['HA_Open'][1:]= (df['HA_Open'].shift(1)+df['HA_Close'].shift(1))/2 
    for i in range(1, len(df)):
        df['HA_Open'][i]=(df['HA_Open'][i-1]+df['HA_Close'][i-1])/2 
    df['HA_High']=df[['HA_Open','HA_Close','high']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','low']].min(axis=1)
    return df


In [19]:
def EMA_special(ema12,ema26):
    my_var=[]
    for i in range(len(ema12)):
        sig1 = ema12[i] - ema26[i]
        my_var.append(sig1)
    return my_var


In [20]:
@jit
def Heiken_ashim1(haopen, hahigh, halow, haclose, ema_diff, olhc4, ema12):
    heikensignal_y = []
    count = 0
    for i in range(len(haopen)):
        if (ema_diff[i] < 0 and ema_diff[i-1] < ema_diff[i]) or (ema_diff[i] > 0 and ema_diff[i-1] < ema_diff[i]):
            if haopen[i] < haclose[i]:
                heikensignal_y.append(1)
            elif haopen[i] > haclose[i]:
                count += 1
                if count <= 2 and olhc4[i] > ema12[i]:
                    heikensignal_y.append(1)
                else:
                    heikensignal_y.append(0)
            else:
                heikensignal_y.append(0)
        elif (ema_diff[i] < 0 and ema_diff[i-1] > ema_diff[i]) or (ema_diff[i] > 0 and ema_diff[i-1] > ema_diff[i]):
            if haopen[i] > haclose[i]:
                heikensignal_y.append(-1)
            elif haopen[i] < haclose[i]:
                count += 1
                if count <= 2 and olhc4[i] < ema12[i]:
                    heikensignal_y.append(-1)
                else:
                    heikensignal_y.append(0)
            else:
                heikensignal_y.append(0)
        else:
            heikensignal_y.append(0)
    
    return heikensignal_y


In [21]:
def calculate_williams_r(df, period):
    """
    Calculate Williams %R for given data and period.
    """
    highest_high = df['high'].rolling(window=period).max()
    lowest_low = df['low'].rolling(window=period).min()
    
    williams_r = (highest_high - df['close']) / (highest_high - lowest_low) * -100
    
    return williams_r


def generate_williams_r_signals(df, period=14, overbought_threshold=-20, oversold_threshold=-80):
    """
    Generate buy/sell signals based on Williams %R indicator.
    """
    df['Williams_R'] = calculate_williams_r(df, period)
    
    signals = pd.DataFrame(index=df.index)
    signals['price'] = df['close']
    
    # Generate signals
    signals['Buy'] = (df['Williams_R'] <= oversold_threshold).astype(int)
    signals['Sell'] = (df['Williams_R'] >= overbought_threshold).astype(int) * 1
    
    return signals

In [22]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0


In [47]:
import numpy as np

def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan


In [24]:
def ascending_triangle_pattern(open, high, low, close, pivot, pointpos,df):
    signals = np.zeros(len(open))
    for candleid in range(0, len(df)):
        maxim = np.array([])
        minim = np.array([])
        xxmin = np.array([])
        xxmax = np.array([])
        for i in range(candleid - backcandles, candleid + 1):
            if df.iloc[i].pivot == 1:
                minim = np.append(minim, df.iloc[i].low)
                xxmin = np.append(xxmin, i)  # could be i instead df.iloc[i].name
            if df.iloc[i].pivot == 2:
                maxim = np.append(maxim, df.iloc[i].high)
                xxmax = np.append(xxmax, i)  # df.iloc[i].name

        if (xxmax.size < 3 and xxmin.size < 3) or xxmax.size == 0 or xxmin.size == 0:
            continue

        slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
        slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

        if abs(rmax) >= 0.9 and abs(rmin) >= 0.9 and slmin >= 0.0001 and slmax <= -0.0001:
            #print(rmin, rmax, candleid)
            signals[candleid] = 1

    return signals


In [25]:
dfd = pd.read_csv("C://Users//sahil//OneDrive//Desktop//iiiiiiiiiiii.csv")
dfd.shape

(157659, 59)

In [26]:
dfd.drop(columns=['pointpos'], inplace=True)

In [27]:
# Drop NaN values
dfd = dfd.dropna()

# Reset index
dfd = dfd.reset_index(drop=True)

In [28]:
pd.set_option('display.max_columns', None)
dfd.tail()

,Date,Open,High,Low,Close,Volume,olhc4,date1,Timestamp,Time,Pivot Point,Bottom Pivot Line,res1,sup1,res2,sup2,top_pivot_line,Upper Bound,Lower Bound,macd,macd_signal,macd_hist,macd_y,supertrend,supertrend_y,rsi14,rsi_y_not_final,dema7,dema21,dema_y,TEMA7,TEMA21,tema_y,rsi_y,ema9,ema12,ema26,ADX,adx_y1,adx_y2,DMplus,DMminus,TR,DIplus,DIminus,adx_di_y,HA_Close,HA_Open,HA_High,HA_Low,ema_diff,ha_olhc4,ha_signals_y,Williams_R,williams_y,pivot,ascending_triangle_pattern_y,col_y
134088,Fri Feb 23 2024 15:15:00 GMT+0530 (India Stand...,758.90,759.00,757.55,757.95,370315,758.3500,02-23-2024,23-02-2024 15:15,15:15:00,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.281431,-0.085505,-0.195926,0,761.819225,-1,37.215506,0,758.558886,759.357769,0,758.174342,759.282069,0,0,759.333405,759.482929,759.764360,9.677419,-1,0,0.00,1.15,1.45,9.677419,23.460411,0,758.3500,759.331102,759.331102,757.55,-0.281431,758.640551,-1,-91.011236,1,0,0.0,0
134089,Fri Feb 23 2024 15:18:00 GMT+0530 (India Stand...,757.90,759.00,757.30,758.10,485920,758.0750,02-23-2024,23-02-2024 15:18,15:18:00,765.65,765.0,774.3,758.3,781.65,749.65,766.3,758.3,749.65,-0.370904,-0.142585,-0.228319,-1,761.805352,-1,38.504311,0,758.243928,759.112431,-1,757.919538,758.951573,-1,0,759.086724,759.270170,759.641074,9.510086,-1,0,0.00,0.25,1.70,9.510086,24.495677,0,758.0750,758.840551,759.000000,757.30,-0.370904,758.303888,-1,-82.978723,1,1,0.0,-1
134090,Fri Feb 23 2024 15:21:00 GMT+0530 (India Stand...,758.35,760.45,758.20,759.25,263506,759.0625,02-23-2024,23-02-2024 15:21,15:21:00,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.345038,-0.183076,-0.161963,-1,761.805352,-1,47.416344,0,758.560894,759.100891,-1,758.489878,758.968213,-1,0,759.119380,759.267067,759.612106,9.668508,-1,0,1.45,0.00,2.35,9.668508,23.480663,0,759.0625,758.457775,760.450000,758.20,-0.345038,759.042569,1,-58.510638,0,0,0.0,-1
134091,Fri Feb 23 2024 15:24:00 GMT+0530 (India Stand...,759.45,759.65,758.50,759.45,201659,759.2625,02-23-2024,23-02-2024 15:24,15:24:00,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.304887,-0.207438,-0.097449,-1,761.805352,-1,48.805878,0,758.869756,759.127297,-1,758.961555,759.036017,-1,0,759.185504,759.295211,759.600098,9.776536,-1,0,0.00,0.00,1.15,9.776536,23.743017,0,759.2625,758.760138,759.650000,758.50,-0.304887,759.043159,1,-54.255319,0,0,0.0,-1
134092,Fri Feb 23 2024 15:27:00 GMT+0530 (India Stand...,759.50,760.35,759.00,760.35,289069,759.8000,02-23-2024,23-02-2024 15:27,15:27:00,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.198160,-0.205582,0.007422,1,761.805352,-1,54.617578,0,759.473506,759.307986,-1,759.761479,759.319733,1,0,759.418403,759.457486,759.655646,12.464589,0,0,0.70,0.00,1.35,12.464589,24.079320,0,759.8000,759.011319,760.350000,759.00,-0.198160,759.540330,1,-22.784810,0,0,0.0,0


In [29]:
dfd = dfd.drop(["date1","Time","adx_y1","Date",],axis=1)

In [30]:
dfd = dfd.rename(columns={'macd': 'MACD_12_26_9', 'macd_signal': 'MACDs_12_26_9', 'macd_hist': 'MACDh_12_26_9','Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume','Timestamp':'datetime'})

In [31]:
df1=dfd

In [32]:
dfd.tail()

,open,high,low,close,volume,olhc4,datetime,Pivot Point,Bottom Pivot Line,res1,sup1,res2,sup2,top_pivot_line,Upper Bound,Lower Bound,MACD_12_26_9,MACDs_12_26_9,MACDh_12_26_9,macd_y,supertrend,supertrend_y,rsi14,rsi_y_not_final,dema7,dema21,dema_y,TEMA7,TEMA21,tema_y,rsi_y,ema9,ema12,ema26,ADX,adx_y2,DMplus,DMminus,TR,DIplus,DIminus,adx_di_y,HA_Close,HA_Open,HA_High,HA_Low,ema_diff,ha_olhc4,ha_signals_y,Williams_R,williams_y,pivot,ascending_triangle_pattern_y,col_y
134088,758.90,759.00,757.55,757.95,370315,758.3500,23-02-2024 15:15,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.281431,-0.085505,-0.195926,0,761.819225,-1,37.215506,0,758.558886,759.357769,0,758.174342,759.282069,0,0,759.333405,759.482929,759.764360,9.677419,0,0.00,1.15,1.45,9.677419,23.460411,0,758.3500,759.331102,759.331102,757.55,-0.281431,758.640551,-1,-91.011236,1,0,0.0,0
134089,757.90,759.00,757.30,758.10,485920,758.0750,23-02-2024 15:18,765.65,765.0,774.3,758.3,781.65,749.65,766.3,758.3,749.65,-0.370904,-0.142585,-0.228319,-1,761.805352,-1,38.504311,0,758.243928,759.112431,-1,757.919538,758.951573,-1,0,759.086724,759.270170,759.641074,9.510086,0,0.00,0.25,1.70,9.510086,24.495677,0,758.0750,758.840551,759.000000,757.30,-0.370904,758.303888,-1,-82.978723,1,1,0.0,-1
134090,758.35,760.45,758.20,759.25,263506,759.0625,23-02-2024 15:21,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.345038,-0.183076,-0.161963,-1,761.805352,-1,47.416344,0,758.560894,759.100891,-1,758.489878,758.968213,-1,0,759.119380,759.267067,759.612106,9.668508,0,1.45,0.00,2.35,9.668508,23.480663,0,759.0625,758.457775,760.450000,758.20,-0.345038,759.042569,1,-58.510638,0,0,0.0,-1
134091,759.45,759.65,758.50,759.45,201659,759.2625,23-02-2024 15:24,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.304887,-0.207438,-0.097449,-1,761.805352,-1,48.805878,0,758.869756,759.127297,-1,758.961555,759.036017,-1,0,759.185504,759.295211,759.600098,9.776536,0,0.00,0.00,1.15,9.776536,23.743017,0,759.2625,758.760138,759.650000,758.50,-0.304887,759.043159,1,-54.255319,0,0,0.0,-1
134092,759.50,760.35,759.00,760.35,289069,759.8000,23-02-2024 15:27,765.65,765.0,774.3,758.3,781.65,749.65,766.3,765.0,758.30,-0.198160,-0.205582,0.007422,1,761.805352,-1,54.617578,0,759.473506,759.307986,-1,759.761479,759.319733,1,0,759.418403,759.457486,759.655646,12.464589,0,0.70,0.00,1.35,12.464589,24.079320,0,759.8000,759.011319,760.350000,759.00,-0.198160,759.540330,1,-22.784810,0,0,0.0,0


In [33]:
dfd.columns

Index(['open', 'high', 'low', 'close', 'volume', 'olhc4', 'datetime',
       'Pivot Point', 'Bottom Pivot Line', 'res1', 'sup1', 'res2', 'sup2',
       'top_pivot_line', 'Upper Bound', 'Lower Bound', 'MACD_12_26_9',
       'MACDs_12_26_9', 'MACDh_12_26_9', 'macd_y', 'supertrend',
       'supertrend_y', 'rsi14', 'rsi_y_not_final', 'dema7', 'dema21', 'dema_y',
       'TEMA7', 'TEMA21', 'tema_y', 'rsi_y', 'ema9', 'ema12', 'ema26', 'ADX',
       'adx_y2', 'DMplus', 'DMminus', 'TR', 'DIplus', 'DIminus', 'adx_di_y',
       'HA_Close', 'HA_Open', 'HA_High', 'HA_Low', 'ema_diff', 'ha_olhc4',
       'ha_signals_y', 'Williams_R', 'williams_y', 'pivot',
       'ascending_triangle_pattern_y', 'col_y'],
      dtype='object')

In [34]:
data= df1[['open', 'high', 'low', 'close', 'volume', 'olhc4', 'datetime',
       'Pivot Point', 'Bottom Pivot Line', 'res1', 'sup1', 'res2', 'sup2',
       'top_pivot_line', 'Upper Bound', 'Lower Bound', 'MACD_12_26_9',
       'MACDs_12_26_9', 'MACDh_12_26_9', 'macd_y', 'supertrend',
       'supertrend_y', 'rsi14', 'rsi_y_not_final', 'dema7', 'dema21', 'dema_y',
       'TEMA7', 'TEMA21', 'tema_y', 'rsi_y', 'ema9', 'ema12', 'ema26', 'ADX',
       'adx_y2', 'DMplus', 'DMminus', 'TR', 'DIplus', 'DIminus', 'adx_di_y',
       'HA_Close', 'HA_Open', 'HA_High', 'HA_Low', 'ema_diff', 'ha_olhc4',
       'ha_signals_y', 'Williams_R', 'williams_y', 'pivot',
       'ascending_triangle_pattern_y', 'col_y']]
data.head()

,open,high,low,close,volume,olhc4,datetime,Pivot Point,Bottom Pivot Line,res1,sup1,res2,sup2,top_pivot_line,Upper Bound,Lower Bound,MACD_12_26_9,MACDs_12_26_9,MACDh_12_26_9,macd_y,supertrend,supertrend_y,rsi14,rsi_y_not_final,dema7,dema21,dema_y,TEMA7,TEMA21,tema_y,rsi_y,ema9,ema12,ema26,ADX,adx_y2,DMplus,DMminus,TR,DIplus,DIminus,adx_di_y,HA_Close,HA_Open,HA_High,HA_Low,ema_diff,ha_olhc4,ha_signals_y,Williams_R,williams_y,pivot,ascending_triangle_pattern_y,col_y
0,299.10,299.60,298.15,299.60,561967,299.1125,02-01-2019 09:15,298.416667,297.275,302.983333,296.133333,305.266667,291.566667,299.558333,299.558333,298.416667,1.086107,1.049181,0.036926,0,298.820699,1,62.626015,0,300.276143,300.215224,0,300.150831,300.650666,0,0,299.711630,299.462078,298.375971,23.711340,0,0.00,2.15,2.55,23.711340,22.680412,0,299.1125,300.266237,300.266237,298.150000,1.086107,299.448743,-1,-43.137255,0,1,0.0,0
1,299.45,302.00,299.45,301.80,889002,300.6750,02-01-2019 09:18,298.416667,297.275,302.983333,296.133333,305.266667,291.566667,299.558333,299.558333,298.416667,1.192156,1.077776,0.114380,0,298.223205,1,73.898517,0,301.020630,300.613701,0,301.121489,301.117402,0,0,300.129304,299.821759,298.629603,40.000000,0,2.40,0.00,2.55,40.000000,18.723404,0,300.6750,299.689368,302.000000,299.450000,1.192156,300.453592,1,-5.194805,-1,0,0.0,0
2,301.75,302.15,301.00,301.50,772001,301.6000,02-01-2019 09:21,298.416667,297.275,302.983333,296.133333,305.266667,291.566667,299.558333,302.983333,299.558333,1.237725,1.109765,0.127959,1,299.005476,1,70.764167,0,301.356864,300.900758,0,301.468292,301.413145,1,0,300.403443,300.079950,298.842225,37.552743,0,0.15,0.00,1.15,37.552743,18.565401,1,301.6000,300.182184,302.150000,300.182184,1.237725,301.028592,1,-16.250000,-1,2,0.0,1
3,301.50,301.70,301.00,301.20,287277,301.3500,02-01-2019 09:24,298.416667,297.275,302.983333,296.133333,305.266667,291.566667,299.558333,302.983333,299.558333,1.235390,1.134890,0.100500,1,299.005476,1,67.673071,0,301.423692,301.090882,1,301.451340,301.566608,-1,0,300.562755,300.252265,299.016875,36.286920,0,0.00,0.00,0.70,36.286920,18.565401,1,301.3500,300.891092,301.700000,300.891092,1.235390,301.208046,0,-23.750000,0,0,0.0,1
4,301.20,301.55,301.10,301.35,267658,301.3000,02-01-2019 09:27,298.416667,297.275,302.983333,296.133333,305.266667,291.566667,299.558333,302.983333,299.558333,1.231448,1.154202,0.077246,1,299.005476,1,68.415956,0,301.512927,301.274944,1,301.492931,301.714245,-1,0,300.720204,300.421147,299.189699,35.983264,0,0.00,0.00,0.45,35.983264,18.410042,1,301.3000,301.120546,301.550000,301.100000,1.231448,301.267637,0,-20.000000,-1,0,0.0,1


In [35]:
data = data.drop(["datetime"],axis=1)

In [36]:
data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'olhc4', 'Pivot Point',
       'Bottom Pivot Line', 'res1', 'sup1', 'res2', 'sup2', 'top_pivot_line',
       'Upper Bound', 'Lower Bound', 'MACD_12_26_9', 'MACDs_12_26_9',
       'MACDh_12_26_9', 'macd_y', 'supertrend', 'supertrend_y', 'rsi14',
       'rsi_y_not_final', 'dema7', 'dema21', 'dema_y', 'TEMA7', 'TEMA21',
       'tema_y', 'rsi_y', 'ema9', 'ema12', 'ema26', 'ADX', 'adx_y2', 'DMplus',
       'DMminus', 'TR', 'DIplus', 'DIminus', 'adx_di_y', 'HA_Close', 'HA_Open',
       'HA_High', 'HA_Low', 'ema_diff', 'ha_olhc4', 'ha_signals_y',
       'Williams_R', 'williams_y', 'pivot', 'ascending_triangle_pattern_y',
       'col_y'],
      dtype='object')

In [37]:
X = data.drop('col_y', axis=1)
y=data["col_y"]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
# Create the random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [40]:
# Make predictions on the test set
y_pred = rf.predict(X_test)

In [41]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [42]:
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy: 0.97
Precision: 0.96
Recall: 0.97
F1-score: 0.96


In [ ]:
from sklearn.model_selection import cross_val_score

# Create the random forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform 5-fold cross-validation
scores = cross_val_score(rf, X, y, cv=5, scoring='accuracy')

# Print the cross-validation scores
print("Cross-Validation Scores:")
for i, score in enumerate(scores):
    print(f"Fold {i+1}: {score:.2f}")

# Calculate the mean and standard deviation of the scores
print(f"\nMean Accuracy: {scores.mean():.2f} +/- {scores.std():.2f}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming your target variable is named 'target'
X = data.drop('col_y', axis=1)
y=data["col_y"]

# Split the data into training, validation, and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Train the random forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train, rf.predict(X_train))
print(f"Training Accuracy: {train_accuracy:.2f}")

# Evaluate the model on the validation set
val_accuracy = accuracy_score(y_val, rf.predict(X_val))
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, rf.predict(X_test))
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
tv = TvDatafeed()

import pandas_ta as ta
def fetch_data():
    btc = tv.get_hist(symbol='SBIN',exchange='NSE',interval=Interval.in_3_minute,n_bars=1000)
    df = pd.DataFrame(btc)
    return df

while True:
    df = fetch_data()
    df.reset_index(inplace=True)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df["olhc4"] = (df["open"] + df["high"] + df["low"] + df["close"])/4
    df = calculate_pivot_points(df)
    
    ####################################################################################################################
    
    df = pivot_getting(df)
    
    ####################################################################################################################
   
    df = res3sup3(df)
    
    ###################################################################################################################
    open_prices = df["open"].values
    res1 = df["res1"].values
    res2 = df["res2"].values
    res3 = df["res3"].values
    top_pivot_line = df["top_pivot_line"].values
    pivot_point = df["Pivot Point"].values
    bottom_pivot_line = df["Bottom Pivot Line"].values
    sup1 = df["sup1"].values
    sup2 = df["sup2"].values
    sup3 = df["sup3"].values
    
    ###################################################################################################################
    
    upper_bound_np, lower_bound_np = identify_price_position_numpy(open_prices, res2, res1, top_pivot_line, pivot_point,bottom_pivot_line, sup1, sup2,res3,sup3)
    result_df = pd.DataFrame({
        "Upper Bound": upper_bound_np,
        "Lower Bound": lower_bound_np})
    
    df = pd.concat([df, result_df], axis=1)
    
    ####################################################################################################################
    
    ma_cd = ta.macd(df["close"], fast=12, slow=26, signal=9)
    df = pd.concat([df,ma_cd],axis=1)
    macd = df["MACD_12_26_9"].values
    macd_signal = df["MACDs_12_26_9"].values
    macd_hist = df["MACDh_12_26_9"].values
    upper_bound = df["Upper Bound"].values
    lower_bound = df["Lower Bound"].values
    close = df["close"].values
    df["macd_y"] = macd_(macd, macd_signal, macd_hist, upper_bound, lower_bound, close) #macd
    
    ####################################################################################################################
    
    su = ta.supertrend(df["high"],df["low"],df["close"],length=14,multiplier=3)
    df["supertrend"]=su["SUPERT_14_3.0"]
    upper_bound = df["Upper Bound"].values
    lower_bound = df["Lower Bound"].values
    close = df["close"].values
    df["supertrend_y"] = supertrend_y(df,lower_bound,upper_bound,close) #supertrend
    
    ###################################################################################################################
        
    df['rsi14'] = ta.rsi(df['close'], length=14)
    rsi_values=df["rsi14"].values
    price_highs = df["high"].values
    rsi_y11 = bearish_divergence_rsi_macdxs(rsi_values, price_highs)
    df['rsi_y_not_final'] = rsi_y11
    sam = []  # empty list to store the result
    prev_start = 0

    ###################################################################################################################
    
    df["dema7"] = ta.dema(df["close"], length=7,talib=False)
    df["dema21"] = ta.dema(df["close"], length=21,talib=False)
    dema7 = df['dema7'].values
    dema21 = df['dema21'].values
    df['dema_y'] = DEMA_Y(dema7,dema21,macd_hist,upper_bound,lower_bound,close,rsi_values,ema_fast,ema_slow,rsi_overbought,rsi_oversold) #DEMA
    
    ###################################################################################################################
    
    df["TEMA7"] = ta.tema(df["close"], length=7,talib=False)
    df["TEMA21"] = ta.tema(df["close"], length=21,talib=False)
    tema_7 = df["TEMA7"].values
    tema_21 = df["TEMA21"].values
    df['tema_y'] = TEMA_Y(tema_7,tema_21,macd_hist,upper_bound,lower_bound,close) #TEMA
    
    ##########################################################################################################
    
    
    while prev_start < len(df):
        i = prev_start
        if df['rsi_y_not_final'][i] == -1:  # check if 'a' is 1
            sam.append(0)
            found_one_in_b = False
            for j in range(i+1, min(i+5, len(df))):  # check next 4 entries in 'b' starting from i+2  (1,5)
                if df['tema_y'][j] == -1:
                    found_one_in_b = True
                    start_index = j
                    break
                else:
                    sam.append(0)
                    prev_start = j+1
            if found_one_in_b:  # if 'b' has 1 within next 4 entries
                for k in range(start_index, len(df)):  # check for continuous 1's in 'b'
                    if df['tema_y'][k] == 0 or df['tema_y'][k] == 1:
                        prev_start = k
                        break
                    sam.append(-1)
                    if k == len(df)-1:prev_start = k
            else:
                prev_start = j+1  
        else:
            sam.append(0)
            prev_start += 1
    df['rsi_y'] = sam
    
    ####################################################################################################################
    
    adxx= ta.adx(df['high'], df['low'], df['close'],length=8)
    df["ADX"] = adxx["ADX_8"]    
    df['ema9'] = ta.ema(df['close'], length=9,talib=False)
    df['ema12'] = ta.ema(df['close'], length=12,talib=False)
    df['ema26'] = ta.ema(df['close'], length=26,talib=False)
    
    adxx2= ta.adx(df['high'], df['low'], df['close'],length=14)
    adx2 = adxx2["ADX_14"]
    ema9 = df['ema9'].values
    ema12 = df['ema12'].values
    ema26 = df['ema26'].values
    adx1 = df["ADX"].values
    high = df['high'].values
    low = df['low'].values
    open = df['open'].values 
    
    
    
    ####################################################################################################################
    
    
    #df['adx_y1'] = calculate_signals_adx1(df) #adx1
    df['adx_y2'] = calculate_signals_adx2(close, ema9, ema26, rsi_values, macd, macd_signal,high,low,adx2)
    
    ######################################################################################################################

    adx_di_signals = generate_signals(df,upper_bound,lower_bound,close)
    df["adx_di_y"] = adx_di_signals["Buy"] - adx_di_signals["Sell"]
    
    ######################################################################################################################
    
    df = HA(df)
    df["ema_diff"] = EMA_special(ema12,ema26)
    haopen = df["HA_Open"].values
    hahigh = df["HA_High"].values
    halow = df["HA_Low"].values
    haclose = df["HA_Close"].values
    ema_diff = df["ema_diff"].values
    df['ha_olhc4'] = (df["HA_Open"] + df["HA_High"] + df["HA_Low"] + df["HA_Close"])/4
    olhc4 = df['ha_olhc4'].values
    ha_signals = Heiken_ashim1(haopen,hahigh,halow,haclose,ema_diff,olhc4,ema12)              
    df["ha_signals_y"] = ha_signals
    
    #################################################### plot ###############################################################
    
    s=generate_williams_r_signals(df, period=14, overbought_threshold=-20, oversold_threshold=-80)
    sm = s["Buy"] - s["Sell"]
    df["williams_y"] = sm
    
    ###################################################################################################################
    
    df['pivot'] = df.apply(lambda x: pivotid(df, x.name,3,3), axis=1)
    df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
    
    pivot = df["pivot"].values
    pointpos = df["pointpos"].values
    backcandles = 20
    
    asc_signals = ascending_triangle_pattern(open, high, low, close, pivot, pointpos,df)
    df["ascending_triangle_pattern_y"] = asc_signals
    
    df11 = df.drop(["res3","sup3","pointpos","symbol"],axis=1)
    
    new_column_order = ['open', 'high', 'low', 'close', 'volume', 'olhc4', 'Pivot Point',
       'Bottom Pivot Line', 'res1', 'sup1', 'res2', 'sup2', 'top_pivot_line',
       'Upper Bound', 'Lower Bound', 'MACD_12_26_9', 'MACDs_12_26_9',
       'MACDh_12_26_9', 'macd_y', 'supertrend', 'supertrend_y', 'rsi14',
       'rsi_y_not_final', 'dema7', 'dema21', 'dema_y', 'TEMA7', 'TEMA21',
       'tema_y', 'rsi_y', 'ema9', 'ema12', 'ema26', 'ADX', 'adx_y2', 'DMplus',
       'DMminus', 'TR', 'DIplus', 'DIminus', 'adx_di_y', 'HA_Close', 'HA_Open',
       'HA_High', 'HA_Low', 'ema_diff', 'ha_olhc4', 'ha_signals_y',
       'Williams_R', 'williams_y', 'pivot', 'ascending_triangle_pattern_y',"datetime"]
    
    df11 = df11.reindex(columns=new_column_order)
    df11 = df11.dropna()
    df11 = df11.reset_index(drop=True)
    datetimex = df11["datetime"]
    df11 = df11.drop("datetime",axis=1)
    final_y = rf.predict(df11)
    final_y = pd.DataFrame(final_y)
    df = pd.concat([datetimex,df11,final_y],axis=1)
    
    #######################################################################################################################
    
    dfx = df.tail(125)
    fig = go.Figure(data=[go.Candlestick(x=dfx['datetime'],
                                     open=dfx['open'],
                                     high=dfx['high'],
                                     low=dfx['low'],
                                     close=dfx['close'],
                                     increasing_line_color='green',
                                     decreasing_line_color='red')])
    
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['Pivot Point'], mode='lines',line=dict(color='blue', dash='dot')))
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['Bottom Pivot Line'], mode='lines',line=dict(color='lightblue', dash='dot')))
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['top_pivot_line'], mode='lines',line=dict(color='lightblue', dash='dot')))
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['res1'], mode='lines',line=dict(color='darkred', dash='dot')))
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['res2'], mode='lines',line=dict(color='pink', dash='dot')))
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['sup1'], mode='lines',line=dict(color='green', dash='dot')))
    fig.add_trace(go.Scatter(x=dfx['datetime'], y=dfx['sup2'], mode='lines',line=dict(color='lightgreen', dash='dot')))

    fig.update_layout(title="Candlestick Chart",
                      xaxis_title="Date",
                      yaxis_title="Price/Volume",
                      xaxis_rangeslider_visible=False)
    
    pio.write_html(fig, 'candlestick_chart.html')
    webbrowser.open_new_tab('candlestick_chart.html')
    
    print(df)
    sleep(180)

               datetime    open    high     low   close    volume     olhc4  \
0   2024-04-05 09:15:00  757.30  757.85  754.00  755.40  476552.0  756.1375   
1   2024-04-05 09:18:00  755.50  755.70  752.60  753.60  342572.0  754.3500   
2   2024-04-05 09:21:00  753.90  755.80  753.30  753.75  325870.0  754.1875   
3   2024-04-05 09:24:00  753.80  756.50  753.00  755.75  294778.0  754.7625   
4   2024-04-05 09:27:00  755.65  758.90  755.10  757.75  413325.0  756.8500   
..                  ...     ...     ...     ...     ...       ...       ...   
870 2024-04-16 15:15:00  752.85  753.00  751.55  752.05  266126.0  752.3625   
871 2024-04-16 15:18:00  752.15  752.35  750.70  752.00  253102.0  751.8000   
872 2024-04-16 15:21:00  752.00  752.25  751.40  751.45  190166.0  751.7750   
873 2024-04-16 15:24:00  751.40  752.80  751.15  752.45  195979.0  751.9500   
874 2024-04-16 15:27:00  752.50  753.00  752.05  752.25  172079.0  752.4500   

     Pivot Point  Bottom Pivot Line        res1    